In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyvista as pv
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

In [3]:
p  = !pwd
root=Path(p[0])

In [4]:
ROOT = root.parent
marv_obj = ROOT / "meshes/marv_lid.obj"
test_mesh = pv.read(marv_obj)

In [ ]:
test_mesh.plot()

In [6]:
import yaml
with open(root.parent / f"robots/marv.yaml", "r") as file:
    robot_params = yaml.safe_load(file)

In [7]:
import torch
driving_part_bboxes = [torch.tensor(bbox) for bbox in robot_params["driving_part_bboxes"]]
body_bbox = torch.tensor(robot_params["body_bbox"])

In [8]:
robot_points = torch.tensor(pv.read(str(marv_obj)).points)

In [9]:
assert test_mesh.is_all_triangles

In [10]:
import pyacvd


def extract_flipper_points_from_mesh(mesh: pv.PolyData, n_points: int = 100, **clus_opts) -> torch.Tensor:
    delaunay = mesh.delaunay_3d()
    surf = delaunay.extract_surface()
    clus: pyacvd.Clustering = pyacvd.Clustering(surf)
    clus.cluster(n_points, **clus_opts)
    return torch.tensor(clus.cluster_centroid)

In [ ]:
from flipper_training.utils.geometry import points_in_oriented_box
driving_meshes = []
driving_points = []
faces = test_mesh.faces.reshape(-1, 4)[:, 1:]
for p in driving_part_bboxes:
    mask = points_in_oriented_box(robot_points[:, :2], p)
    nonzero = mask.nonzero().numpy().flatten()
    pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)
    driving_meshes.append(pointgrid)
    driving_points.append(extract_flipper_points_from_mesh(pointgrid, 200))

In [ ]:
driving_meshes[0].plot(render_points_as_spheres=True)

In [ ]:
delaunay = driving_meshes[0].delaunay_3d()
delaunay.plot()

In [ ]:
surface = delaunay.extract_surface()
surface_points = pv.PolyData(surface.points)
surface_points.plot(render_points_as_spheres=True, point_size=10,)

In [15]:
import pyacvd

In [ ]:
clus = pyacvd.Clustering(surface)
clus.cluster(400, maxiter=1000, iso_try=50)

In [ ]:
pv.PolyData(clus.cluster_centroid).plot(render_points_as_spheres=True, point_size=10,)

In [18]:
body_mask = points_in_oriented_box(robot_points[:, :2], body_bbox)
nonzero = body_mask.nonzero().numpy().flatten()
body_pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)

In [ ]:
body_pointgrid.plot()

In [ ]:
body_delaunay = body_pointgrid.delaunay_3d()
body_delaunay.plot()

In [ ]:
pv.PolyData(pv.voxelize(body_delaunay,density=0.05,check_surface=False).points).plot(render_points_as_spheres=True, point_size=10,)